In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [126]:
#Uploading the data
xls = pd.ExcelFile('The_data.xlsx')

#Age and Sex
df_age_sex = pd.read_excel(xls, 'Age and Sex')
df_age_sex = df_age_sex.drop('YEAR', 1)
df_age_sex.columns = ["Sra", "AGE GROUP", "TOTAL", "MALE", "FEMALE"]


#elevation
df_elevation = pd.read_excel(xls, 'Elevation')
df_elevation['Elevation']=df_elevation['Elevation'].fillna("No Data")


#Marital Status
df_marital_status = pd.read_excel(xls, 'Marital Status')
df_marital_status = df_marital_status.drop('YEAR', 1)


#Educational Attainment
df_educational_attainment = pd.read_excel(xls, 'Educational Attainment')
df_educational_attainment = df_educational_attainment.drop('YEAR', 1)


#Types of Households
df_housing_units_type = pd.read_excel(xls, 'Housing Units Type')
df_housing_units_type = df_housing_units_type.drop('YEAR', 1)


#House Value
df_housing_value = pd.read_excel(xls, 'Housing Value')
df_housing_value = df_housing_value.drop('YEAR', 1)


#House Structure Age
df_year_house_built = pd.read_excel(xls, 'Year Structure Built')
df_year_house_built = df_year_house_built.drop('YEAR', 1)


#Vehicle Data
df_vehicles = pd.read_excel(xls, 'Vehicle Availability')
df_vehicles = df_vehicles.drop('YEAR', 1)

#Population Density
df_pop_density = pd.read_excel(xls, 'Population Density')



In [127]:
#Making a dataframe... with the data..




In [128]:
#The unique regions (SRAs) we have for the census data

regions = df_age_sex["Sra"].unique()
df_regions = pd.DataFrame(regions)
df_regions.columns = ["Region"]

In [129]:
#################################################################
# Our main dataframe that we'll use to create the SIR equations #
#################################################################

df = pd.DataFrame(data=regions, columns=["Region"])

In [130]:
#Adding median age to the df

median_age = df_age_sex[(len(df_age_sex) - 41):]
median_age = median_age["TOTAL"]
median_age.index=df.index

df['Median Age'] = (median_age)

In [131]:
#Adding house median value to the df

median_house_value = df_housing_value[(len(df_housing_value) - 41):]
median_house_value = median_house_value["NUMBER"]
median_house_value.index=df.index

df['Median House Value'] = (median_house_value)

In [132]:
#Population density...

df_pop_density = df_pop_density.sort_values("Neighborhood")

df['Population Density'] = ("n/a")


In [163]:
pd.options.mode.chained_assignment = None  # default='warn'


common_sras = []
common_sras_indices = []

for index, row in df.iterrows():
    if(df_pop_density['Neighborhood'].str.contains(row["Region"]).any()):
        common_sras.append(row["Region"])
        common_sras_indices.append(index)
        print("yes")


for i in range(0,len(common_sras)):
    df["Population Density"][common_sras_indices[i]] = df_pop_density[df_pop_density["Neighborhood"] == common_sras[i]]
    x = df["Population Density"][common_sras_indices[i]]
    df["Population Density"][common_sras_indices[i]] = int(x["People per square mile"])

        

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [161]:
#fuck.index = df.index

In [162]:
df

,Region,Median Age,Median House Value,Population Density
0,Alpine,42.3,612870,n/a
1,Anza-Borrego Springs,55.3,306852,n/a
2,Carlsbad,40.8,613993,2840
3,Central San Diego,34.4,521767,n/a
4,Chula Vista,33.9,359311,5000
5,Coastal,35.4,969973,n/a
6,Coronado,31.3,1275412,2860
7,Del Mar-Mira Mesa,37.0,639308,n/a
8,El Cajon,34.8,421170,6950
9,Elliott-Navajo,37.4,516103,n/a
